# Assignment 4: Correlation Analysis and Bootstrapping

## Objective

The field of statistics is divided into two major parts: descriptive and inferential. In this assignment, we will cover two important topics in statistics: correlation analysis and bootstrapping, where the former belongs to the descriptive part and the latter belongs to the inferential part. After this assignment, you will be able to answer the following questions:

1. How to visualize the relationship between two variables?  
2. What is Pearson's correlation? How to compute it?
3. What is Spearman's rank correlation? How to compute it?
4. What's the difference between Pearson's correlation and Spearman's rank correlation? 
5. How to estimate a population parameter based on a sample?
6. How to use the bootstrap to quantify the uncertainty of an estimated value?

**Requirements:**
* Use [pandas](https://pandas.pydata.org/) to manipulate data.
* Use [matplotlib](https://matplotlib.org/) or [seaborn](https://seaborn.pydata.org) to make plots.
* Please pay attention to visualization principles. 


The data can be downloaded from [A4-data.zip](A4-data.zip).


## Part 1. Correlation Analysis

As a data scientist, you often face this kind of question: "Are A and B correlated?" For example, 

* Do Canadian Currency and Oil Price move together?
* Do Vancouver Housing Price and US Stock Market have any correlation?
* Are GPA and Gender independent? 

To answer these questions, you need to conduct a correlation analysis. 

Imagine you are a data scientist working at a real-estate company. You download a property_tax_report from this [webpage](https://data.vancouver.ca/datacatalogue/propertyTax.htm). The dataset contains information on properties from BC Assessment (BCA) and City sources in 2019.  You can find the schema information of the dataset from this [webpage](http://data.vancouver.ca/datacatalogue/propertyTaxAttributes.htm). 

You may think that for a newly built house, it tends to have a higher price than the ones built decades ago. In this assignment, your first job is to figure out whether YEAR_BUILT and HOUSE_PRICE are correlated. 

We first load the data as a DataFrame. 

In [ ]:
import pandas as pd

df = pd.read_csv("property_tax_report_2019.csv")


df['HOUSE_PRICE'] = df.apply(lambda x: (x['CURRENT_LAND_VALUE'] \
                                        +x['CURRENT_IMPROVEMENT_VALUE'])/1000000.0, axis = 1)

### Task A. Visualizations

Since the housing price varies a lot by locations, we will only consider the houses whose postcode starts with 'V6A'. Furthermore, we remove the houses that were built before 1900.

In the following, please make two subplots in one row. For the left subplot, it is a scatter plot with X = `YEAR_BUILT` and Y = `HOUSE_PRICE`; for the right subplot, it is a hexbin plot (gridsize = 20) with X = `YEAR_BUILT` and Y = `HOUSE_PRICE`.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

df = df.query('LEGAL_TYPE == "LAND" & PROPERTY_POSTAL_CODE.str.startswith("V6A", na=False) & YEAR_BUILT >= 1900')
fig, ax = plt.subplots(figsize=(12,5), nrows=1, ncols=2)
ax[0].set_xlabel('Year Built')
ax[0].set_ylabel('House Price')
ax[0].scatter(df['YEAR_BUILT'], df['HOUSE_PRICE'], alpha=.2, marker='D', s=30)
ax[1].set_xlabel('Year Built')
ax[1].set_ylabel('House Price')
ax[1].hexbin(df['YEAR_BUILT'], df['HOUSE_PRICE'], gridsize=20, mincnt=1)

Please write down the **two** most interesting findings that you draw from the plot.

**Findings**
1. [ADD TEXT]
2. [ADD TEXT]

The above plots provide a general impression of the relationship between variables. There are some other visualizations that can provide more insights. One option is to bin one variable and plot percentiles of the other. 


In the following, please make three subplots in a row, where each subplot is a scatter plot with X = YEAR_BUILT and Y = HOUSE_PRICE. 
* The first subplot shows how the 25th percentile of `HOUSE_PRICE` changes over years (X = `YEAR_BUILT`, Y = `25TH_HOUSE_PRICE`); 
* The second subplot shows how the 50th percentile of `HOUSE_PRICE` changes over years (X = `YEAR_BUILT`, Y = `50TH_HOUSE_PRICE`); 
* The third subplot shows how the 75th percentile of `HOUSE_PRICE` changes over years (X = `YEAR_BUILT`, Y = `75TH_HOUSE_PRICE`);  


In [ ]:
dfcor = pd.DataFrame({'YEAR_BUILT': list(df.groupby('YEAR_BUILT').groups.keys())})
dfcor['25TH_HOUSE_PRICE'] = df.groupby('YEAR_BUILT').HOUSE_PRICE.quantile(q=.25, interpolation='linear').to_list()
dfcor['50TH_HOUSE_PRICE'] = df.groupby('YEAR_BUILT').HOUSE_PRICE.quantile(q=.5, interpolation='linear').to_list()
dfcor['75TH_HOUSE_PRICE'] = df.groupby('YEAR_BUILT').HOUSE_PRICE.quantile(q=.75, interpolation='linear').to_list()

fig, ax = plt.subplots(figsize=(20,5), nrows=1, ncols=3)
ax[0].set_xlabel('Year Built')
ax[0].set_ylabel('25th House Price')
ax[0].scatter(dfcor['YEAR_BUILT'], dfcor['25TH_HOUSE_PRICE'], alpha=.5, marker='D', s=50)
ax[1].set_xlabel('Year Built')
ax[1].set_ylabel('50th House Price')
ax[1].scatter(dfcor['YEAR_BUILT'], dfcor['50TH_HOUSE_PRICE'], alpha=.5, marker='D', s=50)
ax[2].set_xlabel('Year Built')
ax[2].set_ylabel('75th House Price')
ax[2].scatter(dfcor['YEAR_BUILT'], dfcor['75TH_HOUSE_PRICE'], alpha=.5, marker='D', s=50)

Please write down the **two** most interesting findings that you draw from the plot.

**Findings**
1. [ADD TEXT]
2. [ADD TEXT]

### Task B. Correlation Coefficient

A correlation coefficient is a numerical measure of some type of correlation, meaning a statistical relationship between a pair of variables. 

In the following, please implement `calc_pearson()` and `calc_spearman()`, respectively. Note that you are *NOT* allowed to use [corr](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.corr.html) from Pandas or [pearsonr](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html#scipy.stats.pearsonr)/[spearmanr](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.spearmanr.html#scipy.stats.spearmanr) from scipy.stats to do this task. In other words, you need to implement the actual algorithms for pearson and spearman.

In [ ]:
def calc_pearson(df, x, y):
    return df[x].cov(df[y]) /(df[x].std() * df[y].std())

def calc_spearman(df, x, y):
    rankx = df[x].rank()
    ranky = df[y].rank()
    return rankx.cov(ranky) / (rankx.std() * ranky.std())

Then, you can use these two functions to compute the Pearson's correlation as well as Spearman's rank correlation for three pairs of variables: <`25TH_HOUSE_PRICE`, `YEAR_BUILT`>, <`50TH_HOUSE_PRICE`, `YEAR_BUILT`>, and <`75TH_HOUSE_PRICE`, `YEAR_BUILT`>. 

In [ ]:
print(dfcor.head(10))
print()

for TH in ["25TH", "50TH", "75TH"]:
    print(TH+"_HOUSE_PRICE\t pearson=%f\t spearman=%f"  \
          %(calc_pearson(dfcor, "YEAR_BUILT", TH+"_HOUSE_PRICE"), \
            calc_spearman(dfcor, "YEAR_BUILT", TH+"_HOUSE_PRICE")))

<img src="correlation-result.png",align="left",width=500,height=500>

Please write down the **two** most interesting findings that you draw from the result.

 

**Findings**
1. 
2. [ADD TEXT]

## Part 2. Bootstrapping

In reality, it is more often than not that you can only collect a sample of the data. Whenever you derive a conclusion from a sample (e.g., Vancouver's housing price has increased by 10% since last year), you should ALWAYS ask yourself: **"CAN I TRUST IT?"**. In other words, you want to know that if the same analysis was conducted on the full data, would the same conclusion be derived? In Part 2, you will learn how to use bootstrapping to answer this question. 

In [ ]:
df_sample = pd.read_csv("property_tax_report_2019_sample.csv")

df_sample['CURRENT_PRICE'] = df_sample.apply(lambda x: x['CURRENT_LAND_VALUE'] \
                                             +x['CURRENT_IMPROVEMENT_VALUE'], axis = 1)

df_sample['PREVIOUS_PRICE'] = df_sample.apply(lambda x: x['PREVIOUS_LAND_VALUE'] \
                                              +x['PREVIOUS_IMPROVEMENT_VALUE'], axis = 1)

df_sample = df_sample[df_sample['LEGAL_TYPE'] == 'STRATA']

### Task 1. Analysis Result Without Bootstrapping

Please compute the median of PREVIOUS_PRICE and CURRENT_PRICE, respectively, and compare them in a bar chart.

In [ ]:
plt.bar(['Previous Price', 'Current Price'], [df_sample['PREVIOUS_PRICE'].median(), df_sample['CURRENT_PRICE'].median()], width=.9, align='center')
plt.yscale('log')

### Task 2. Analysis Result With Bootstrapping

From the above chart, we find that the median of PREVIOUS_PRICE is about 0.77 M, and the median of CURRENT_PRICE is about 0.72 M. Since the numbers were obtained from the sample, **"CAN WE TRUST THESE NUMBERS?"**

In the following, please implement the bootstrap by yourself, and compute a 95%-confidence interval for each number. [This document](./MIT18_05S14_Reading24.pdf) gives a good tutorial about the bootstrap. You can find the description of the algorithm in Section 7.

In [ ]:
median_pre = df_sample['PREVIOUS_PRICE'].median()
median_cur = df_sample['CURRENT_PRICE'].median()
delta_pre = []
delta_cur = []

for i in range(1000):
    med_pre = df_sample['PREVIOUS_PRICE'].sample(n=30, replace=True).median()
    med_cur = df_sample['CURRENT_PRICE'].sample(n=30, replace=True).median()
    delta_pre.append(med_pre)
    delta_cur.append(med_cur)
    if i > 0:
        delta_pre[i-1] = delta_pre[i] - delta_pre[i-1]
        delta_cur[i-1] = delta_cur[i] - delta_cur[i-1]

left_pre = median_pre - sorted(delta_pre)[24]
right_pre = median_pre - sorted(delta_pre)[974]
left_cur = median_cur - sorted(delta_cur)[24]
right_cur = median_cur - sorted(delta_cur)[974]

print(f'95% CI for PREVIOUS_PRICE is: [{left_pre}, {right_pre}]')
print(f'95% CI for CURRENT_PRICE is: [{left_cur}, {right_cur}]')

## Submission

Complete the code in this notebook, and submit it to the CourSys activity Assignment 4.